In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time

import re

In [ ]:
import json

In [2]:
from bs4 import BeautifulSoup

In [3]:
# 웹드라이버 접속
driver = webdriver.Chrome(executable_path=r'C:\Users\jennie\Desktop\BOAZ\chromedriver.exe')

In [ ]:
# 접속할 url 불러오기
df = pd.read_csv('노원_url수집_5개.csv'); 
df = df.iloc[:,2:];df.head()

In [5]:
# 수집할 정보들
rating_list = []
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# rating_df : UserID, ItemID, Rating, Timestamp
# user_review_id : {'공릉역 공릉순두부':{'userid':'reviewid'}, '공릉역 찬이네':{'식당id':'userid','userid',,,},,,}
# review_json : {'공릉역 공릉순두부':{'userid':'review내용'},{'userid':'review내용'},{'userid':'review내용'},,,}
# image_json

for i in range(len(df)):
    
    print('======================================================')
    print(str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
            # driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            # xpath 에러 ㅠㅠ
            driver.find_element_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(4) > div._2kAri > a').click()
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(2)
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['법정동명'][i]+' '+df['상호명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = "날짜 정보 모르것는디"
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = "리뷰 내용 없음"
            print('리뷰 내용 : '+review_content)
            
            
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})
            print('sliced_soup : '+str(sliced_soup))
            if (sliced_soup != None):
                sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                #img_url = re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                try:
                    img_url = re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                except IndexError:
                    img_url = re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                
            else:
                img_url = '이미지가 없는 리뷰'
                
            print('이미지 url : '+img_url)
            print('----------------------------------------------')
            print('\n')

            
            #리뷰정보
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            review_json[restaurant_name][review_id] = review_content
            
            # image_json
            image_json[restaurant_name][review_id] = img_url
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
            
            
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            #review_list.append(리뷰정보)
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

0번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 공릉동 미가족발
식당 구분 : 족발,보쌈
----------------------------------------------
리뷰 총 개수 : 0


1번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 상계동 파란파랑
식당 구분 : 카페,디저트
----------------------------------------------
리뷰 총 개수 : 25
user_url = https://m.place.naver.com/my/5d913bae8f87a842bc7ce85e/review
user_code = 5d913bae8f87a842bc7ce85e
review_id = 36215455_5d913bae8f87a842bc7ce85e
rating = 5
date = 2021.04.25
리뷰 내용 : 저렴하고 커피가 맛있어요
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.place.naver.com/my/5bc6b15043676538261bd355/review
user_code = 5bc6b15043676538261bd355
review_id = 36215455_5bc6b15043676538261bd355
rating = 5
date = 2021.04.09
리뷰 내용 : 좋아요
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.place.naver.com/my/5e6de5658f87a842bc9765ac/review
user_code = 5e6de5658f87a842bc9765ac
review_id = 36215455_5e6de5658f87a842bc9765ac
rating = 4.5
date = 2021.04.02


-더보기 버튼 모두 클릭 완료-
식당 이름 : 공릉동 전주백반
식당 구분 : 백반,가정식
----------------------------------------------
리뷰 총 개수 : 6
user_url = https://m.place.naver.com/my/5e26231e8f87a842bc7f5d75/review
user_code = 5e26231e8f87a842bc7f5d75
review_id = 20919106_5e26231e8f87a842bc7f5d75
rating = 5
date = 2020.10.16
리뷰 내용 : 점심 고민하지 마세요^^
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.place.naver.com/my/5e92b2278f87a842bc0cacba/review
user_code = 5e92b2278f87a842bc0cacba
review_id = 20919106_5e92b2278f87a842bc0cacba
rating = 4
date = 2020.05.08
리뷰 내용 : 괜찮아요
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.place.naver.com/my/5c210660b51390e1c13c1453/review
user_code = 5c210660b51390e1c13c1453
review_id = 20919106_5c210660b51390e1c13c1453
rating = 5
date = 2020.02.09
리뷰 내용 : 맛있어요
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.pla

-더보기 버튼 모두 클릭 완료-
식당 이름 : 월계동 어촌
식당 구분 : 생선회
----------------------------------------------
리뷰 총 개수 : 15
user_url = https://m.place.naver.com/my/5ea7e2fb8f87a842bc71baa1/review
user_code = 5ea7e2fb8f87a842bc71baa1
review_id = 1048323668_5ea7e2fb8f87a842bc71baa1
rating = 3.5
date = 2021.05.02
리뷰 내용 : 굿
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.place.naver.com/my/5c05248328108e5cbcbe19f5/review
user_code = 5c05248328108e5cbcbe19f5
review_id = 1048323668_5c05248328108e5cbcbe19f5
rating = 4.5
date = 2021.04.24
리뷰 내용 : 굿
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://m.place.naver.com/my/5d591be3dbb792042d7d1812/review
user_code = 5d591be3dbb792042d7d1812
review_id = 1048323668_5d591be3dbb792042d7d1812
rating = 4.5
date = 2021.01.04
리뷰 내용 : 친절하고 서비스가 좋아요~^^
sliced_soup : None
이미지 url : 이미지가 없는 리뷰
----------------------------------------------


user_url = https://

In [19]:
import json

In [ ]:
# Sample rating.csv :
rating_df = pd.DataFrame(rating_list)
rating_df.columns = ['UserID','ItemID','Rating','Timestamp']

In [ ]:
# Sample user_review_id.json :

file_path = "./user_review_id.json"

with open(file_path,'w') as outfile:
    json.dump(user_review_id, outfile)

In [ ]:
# Sample review.json :
file_path = "./review.json"

with open(file_path,'w') as outfile:
    json.dump(review_json, outfile)

In [ ]:
# Sample image.json :
file_path = "./image.json"

with open(file_path,'w') as outfile:
    json.dump(image_json, outfile)

In [1]:
## 중간에 멈췄을 때, 다시 돌리기 용
## range에 시작할 index부터 넣어주면 됨


driver = webdriver.Chrome(executable_path=r'C:\Users\jennie\Desktop\BOAZ\chromedriver_win32\chromedriver.exe')

for i in range(len(df)):
    
    print('======================================================')
    print(str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
            # driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            # xpath 에러 ㅠㅠ
            driver.find_element_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(4) > div._2kAri > a').click()
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(2)
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['법정동명'][i]+' '+df['상호명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # 주의!!! 사진 리뷰 유무에 따라 날짜 파싱코드 다름
            # ('span', attrs = {'class':'_3WqoL'})
            # 사진 없는 경우 : 총 6개 중 4번째
            # 사진 있는 경우 : 총 5개 중 3번째
            
            # date
            # 사진 리뷰 없음
            if len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 5:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[2].text
            elif len(one_review[i].find_all('span', attrs = {'class':'_3WqoL'})) == 6:
                date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            else:
                date = "날짜 정보 모르것는디"
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = "리뷰 내용 없음"
            print('리뷰 내용 : '+review_content)
            
            
            # image 내용
            sliced_soup = one_review[i].find('div', attrs = {'class':'_1aFEL _2GO1Q'})
            print('sliced_soup : '+str(sliced_soup))
            if (sliced_soup != None):
                sliced_soup = sliced_soup.find('div',attrs={'class':'dRZ2X'})
                #img_url = re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                try:
                    img_url = re.findall(r'src=(.*jpeg)', str(sliced_soup))[0]
                except IndexError:
                    img_url = re.findall(r'src=(.*jpg)', str(sliced_soup))[0]
                
            else:
                img_url = '이미지가 없는 리뷰'
                
            print('이미지 url : '+img_url)
            print('----------------------------------------------')
            print('\n')

            
            #리뷰정보
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id
            
            # review_json
            review_json[restaurant_name][review_id] = review_content
            
            # image_json
            image_json[restaurant_name][review_id] = img_url
            
            # rating_df_list
            naver_review = user_code, restaurant_name, rating, date
            rating_list.append(naver_review)
            
            
    
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            #review_list.append(리뷰정보)
            
            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

NameError: name 'df' is not defined